# Growth model turned into a traditional model with biomass reaction
Biomass reaction calculated from the concentrations at the maximum growth rate (calculated with the script 'calculate_EGMs_EGVs.ipynb').
EFMs were calculated without or with constraints (capacity constraints on enzymes + dry mass constraint).

In [1]:
import efmtool
import numpy as np
import pandas as pd
from itertools import compress

In [2]:
def make_nice_results(egms, rxns, drop_slack = True):
    """Convert output of efmtool to a data frame
    normalize by the C (constraint) column
    if drop_slack = True, drop columns with slack variables and constraints (named S* or C)"""
    
    # solve "ValueError: Big-endian buffer not supported on little-endian compiler"
    egms = egms.byteswap().newbyteorder()

    res = pd.DataFrame(egms.T, index = ["EFM%s" % (i+1) for i in range(egms.shape[1])], columns = rxns)

    # normalize values by the column C
    if "C" in res.columns:
        res = res.div(res.C, axis = 0)
        
    if drop_slack:
        cols = [c for c in res.columns if c.startswith("S")]
        if "C" in res.columns:
            cols = cols+["C"]
        res = res[res.columns.drop(cols)]
    return res

In [3]:
# stoichiometry
nL = 7   
nR = 7536*3  # multiplied by 3 to take into account the RNA mass of the ribosome
nI = 646
nE = 325

# kinetic parameters [1/h]
k_cat = 79*3600

# dry mass constraint
MW_Glc = 0.18  # glucose MW [g/mmol]
MW_NH4 = 0.018  # ammonium MW [g/mmol]

In [18]:
# read concentrations and select one EGV at the max. growth rate
conc = pd.read_csv("../data/concentrations_LDs_min_lip_mmol.csv", index_col=0)
one_BM = conc.loc[(conc.EGVs == "EGV5") & (conc.mu_str == 1.2637)]

# calculate protein content by summing up all the proteins
protein = one_BM.IG*nI + one_BM.IN*nI + one_BM.EAA*nE + one_BM.ELD*nE + one_BM.EL*nE + one_BM.R*nR + one_BM.AA

## Stoich. matrices without or with constraints (enzyme capacity + dry mass)

In [30]:
rows = ["G", "N", "AA", "LD", "L"]
rxns = ["IG", "IN", "EAA", "ELD", "EL", "BM"] 
matrix = np.array([[1, 0, -1, -nL, 0, -float(one_BM.G)],
                  [0, 1, -1, 0, 0, -float(one_BM.N)],
                  [0, 0, 1, 0, 0, -float(protein)],
                  [0, 0, 0, 1, -1, -float(one_BM.LD)],
                  [0, 0, 0, 0, 1, -float(one_BM.L)]
                  ])
# reversibilities - all irreversible
revs = [0] * matrix.shape[1]

rows2 = rows + ["capIG", "capIN", "capEAA", "capELD", "capEL", "DM"]
rxns2 = rxns[:5] + ["S_IG", "S_IN", "S_EAA", "S_ELD", "S_EL", "BM", "C"] 
matrix2 = np.array([[1, 0, -1, -nL, 0, 0, 0, 0, 0, 0, -float(one_BM.G), 0],
                  [0, 1, -1, 0, 0, 0, 0, 0, 0, 0, -float(one_BM.N), 0],
                  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -float(protein), 0],
                  [0, 0, 0, 1, -1, 0, 0, 0, 0, 0, -float(one_BM.LD), 0],
                  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, -float(one_BM.L), 0],
                  [-1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, k_cat*float(one_BM.IG)],
                  [0, -1, 0, 0, 0, 0, -1, 0, 0, 0, 0, k_cat*float(one_BM.IN)],
                  [0, 0, -1, 0, 0, 0, 0, -1, 0, 0, 0, k_cat*float(one_BM.EAA)],
                  [0, 0, 0, -1, 0, 0, 0, 0, -1, 0, 0, k_cat*float(one_BM.ELD)],
                  [0, 0, 0, 0, -1, 0, 0, 0, 0, -1, 0, k_cat*float(one_BM.EL)],
                  [MW_Glc, MW_NH4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -float(one_BM.mu)]
                  ])

revs2 = [0] * matrix2.shape[1]

## Calculate EFMs

In [31]:
options = efmtool.get_default_options()
options["arithmetic"] = "fractional"

In [32]:
# without constraints
egms = efmtool.calculate_efms(stoichiometry = matrix, 
                              reversibilities = revs, 
                              reaction_names = rxns, 
                              metabolite_names = rows,
                              options = options)
egms_nice = make_nice_results(egms, rxns, True)
egms_nice

,IG,IN,EAA,ELD,EL,BM
EFM1,82.325407,75.325407,75.325407,1.0,1.0,16.174431


In [33]:
# with constraints
egms2 = efmtool.calculate_efms(stoichiometry = matrix2, 
                              reversibilities = revs2, 
                              reaction_names = rxns2, 
                              metabolite_names = rows2,
                              options = options)
egms_nice2 = make_nice_results(egms2, rxns2, True)
egms_nice2

,IG,IN,EAA,ELD,EL,BM
EFM1,6.432061,5.885153,5.885153,0.07813,0.07813,1.263704


## Compare with EGVs

In [17]:
fluxes = pd.read_csv("../data/EGVs_LDs_min_lip.csv", index_col=0)
one_EGV = fluxes.loc[(fluxes.EGVs == "EGV5") & (fluxes.mu_str == 1.2637)]
one_EGV

,IG,IN,EAA,ELD,EL,R_IG,R_IN,R_EAA,R_ELD,R_EL,...,S6,S7,S8,S9,S10,S11,C,mu,mu_str,EGVs
1.2637EGV5,6.432061,5.885153,5.885153,0.07813,0.07813,0.000029,0.000026,0.000035,3.471618e-07,3.471618e-07,...,0.0,2.047904,0.0,0.0,0.0,1.821890e-08,1.0,1.263704,1.2637,EGV5
